In [145]:
from __future__ import division
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

In [146]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [147]:
ratings_train = pd.read_csv('./homework4/data/ratings_train.csv', header=None)
ratings_test = pd.read_csv('./homework4/data/ratings_test.csv', header=None)
ratings_fake = pd.read_csv('./homework4/data/ratings_fake.csv', header=None)

In [148]:
ratings_train.columns = ['user', 'movie', 'rating']
ratings_test.columns = ['user', 'movie', 'rating']
ratings_fake.columns = ['user', 'movie', 'rating']

In [149]:
A = ratings_fake.pivot(index='user', columns='movie').as_matrix()
# A = ratings_train.pivot(index='user', columns='movie').as_matrix()

In [150]:
def optimize_U(U, V, B, C, mu, A):
    Uis = []
    for i in range(U.shape[0]):
        right = 0, 
        left = 0
        for j in range(V.shape[0]):
            right += (B[i] + C[j] + mu - A[i,j]) * V[j]
            left += np.outer(V[j], V[j].T)
        Ui = -np.dot(np.linalg.inv(left), right)
        Uis.append(Ui)
    return np.array(Uis)

In [151]:
def optimize_V(U, V, B, C, mu, A):
    Vjs = []
    for j in range(V.shape[0]):
        right = 0, 
        left = 0
        for i in range(U.shape[0]):
            right += (B[i] + C[j] + mu - A[i, j]) * U[i]
            left += np.outer(U[i], U[i].T)
        Vj = -np.dot(np.linalg.inv(left), right)
        Vjs.append(Vj)
    return np.array(Vjs)

In [152]:
def optimize_B(U, V, B, C, mu, A):
    Bis = []
    for i in range(U.shape[0]):
        b = 0
        for j in range(V.shape[0]):
            b += -(np.dot(U[i], V[j]) + B[i] + C[j] + mu - A[i,j]) / V.shape[0]
        Bis.append(b)
    return np.array(Bis)

In [153]:
def optimize_C(U, V, B, C, mu, A):
    Cjs = []
    for j in range(V.shape[0]):
        c = 0
        for i in range(U.shape[0]):
            c += -(np.dot(U[i], V[j]) + B[i] + C[j] + mu - A[i,j]) / U.shape[0]
        Cjs.append(c)
    return np.array(Cjs)

In [154]:
def update_params(B, U, C, V):
    B = optimize_B(U, V, B, C, mu, A)
    U = optimize_U(U, V, B, C, mu, A)
    C = optimize_C(U, V, B, C, mu, A) 
    V = optimize_V(U, V, B, C, mu, A)
    return (B, U, C, V)

In [172]:
def log_likelihood(B, U, C, V, mu, A):
    lls = 0
    for i in range(U.shape[0]):
        for j in range(V.shape[0]):
            lls += (np.dot(U[i], V[j]) + B[i] + C[j] + mu - A[i,j])
    return lls.sum()

In [173]:
mu = A.mean()
m = A.shape[0]
n = A.shape[1]
k = 5
T = 20
B = np.zeros(m)
C = np.zeros(n)
U = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (m))
V = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (n))

In [174]:
for t in range(T):
    print log_likelihood(B, U, C, V, mu, A)
    B, U, C, V = update_params(B, U, C, V)

-61.0142065975
50.5161824751
4758.74882525
145.797502227
115.70217602
29.8848746375
9.36832872044
5.91552389811
4.70667783293
4.36287500651
4.21930931994
4.15844232135


KeyboardInterrupt: 